![Stack Auto Encoder](https://pbs.twimg.com/media/FdHwrRPWAAAvZDn?format=jpg&name=medium)

The basic structure of Autoencoders contains three layers. The basic purpose of Autoencoders is that when trained, the output should be exactly the same as the input. So if we feed the network a picture of a cat, it will give the exact same picture back. Autoencoders have an input layer, an output layer and one or more hidden layers connecting them, but with the output layer having the same number of nodes as the input layer, and with the purpose of reconstructing its own inputs instead of predicting the target value Y given inputs X.

[Click to see my complete article on Auto Encoder](https://medium.com/machine-learning-researcher/auto-encoder-d942a29c9807)

![Machine Learning Project](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## <font color = #950CDF> Part 1: </font> <font color = #4854E8> Data Preprocessing </font>

![ML](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

#### <font color = blue>Import the Libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

#### <font color = blue> Importing the dataset

In [2]:
movies = pd.read_csv('Dataset/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('Dataset/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('Dataset/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [3]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


#### <font color = blue> Preparing the training set and the test set

In [9]:
training_set = pd.read_csv('Dataset/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('Dataset/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [10]:
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [11]:
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

#### <font color = blue> Getting the number of users and movies

In [12]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

#### <font color = blue> Converting the data into an array with users in lines and movies in columns

In [5]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users +1):
        id_movies = data[:,1][data[:,0] == id_users ]
        id_ratings = data[:,2][data[:,0] == id_users ]
        ratings = np.zeros(nb_movies)
        ratings[id_movies -1] = id_ratings
        new_data.append(list(ratings))
    return new_data

#### <font color = blue> Converting the data into Torch tensors

In [6]:
training_set = convert(training_set)
test_set = convert(test_set)

In [7]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

![Machine Learning Project](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## <font color = #950CDF> Part 2: </font> <font color = #4854E8> Building the Auto Encoder

![ML](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

#### <font color = blue> Creating the architecture of the Neural Network

In [8]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

![Machine Learning Project](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## <font color = #950CDF> Part 3: </font> <font color = #4854E8> Training and Testing the Model </font>

![ML](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

#### <font color = blue> Training the Model

In [9]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data[0]*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: 1.7711133303985094
epoch: 2 loss: 1.096632573171366
epoch: 3 loss: 1.0533801676789927
epoch: 4 loss: 1.0385023174505277
epoch: 5 loss: 1.0307139189022823
epoch: 6 loss: 1.026517515543078
epoch: 7 loss: 1.0237608601205421
epoch: 8 loss: 1.021792910761072
epoch: 9 loss: 1.020784433517058
epoch: 10 loss: 1.0197795504402403
epoch: 11 loss: 1.018616674247378
epoch: 12 loss: 1.0182621153955644
epoch: 13 loss: 1.0178150443388185
epoch: 14 loss: 1.017546251986481
epoch: 15 loss: 1.0170252043758132
epoch: 16 loss: 1.0169583348459938
epoch: 17 loss: 1.0163553127306588
epoch: 18 loss: 1.0167324061978906
epoch: 19 loss: 1.0162972453583903
epoch: 20 loss: 1.0160393731851949
epoch: 21 loss: 1.0161661002519986
epoch: 22 loss: 1.0158637064411984
epoch: 23 loss: 1.0155812994935154
epoch: 24 loss: 1.0158356033417313
epoch: 25 loss: 1.015608623373496
epoch: 26 loss: 1.0156063088067577
epoch: 27 loss: 1.0153759484798945
epoch: 28 loss: 1.014920415593699
epoch: 29 loss: 1.013352768490542
epo

#### <font color = blue> Test the Model

In [11]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data[0]*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: 0.9496440007316151


![Machine Learning Project](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

<b>©</b>Amir Ali